In [1]:
import pandas as pd
from collections import defaultdict
import dvc.api
from tqdm import tqdm
import numpy as np
from functools import reduce

# get original data

In [2]:
df_original = pd.concat([ 
    # pd.read_csv(dvc.api.get_url('datasets/data/wish_attr_extract_label/appen/input_batch_processed/appen_product_attribution_batch1.csv', 
    #     repo='git@github.com:ContextLogic/multitask-llm-rnd.git')), 
    # pd.read_csv(dvc.api.get_url('datasets/data/wish_attr_extract_label/appen/input_batch_processed/appen_product_attribution_batch2_highimpression.csv',
    #     repo='git@github.com:ContextLogic/multitask-llm-rnd.git')), 
    pd.read_csv(dvc.api.get_url('datasets/data/wish_attr_extract_label/appen/input_batch_processed/appen_product_attribution_batch2.csv', 
        repo='git@github.com:ContextLogic/multitask-llm-rnd.git'))
])

In [3]:
len(df_original)

51296

In [4]:
df_original = df_original.dropna(subset=['label_ordering'])

In [5]:
len(df_original)

51296

In [6]:
df_original['label_ordering'] = df_original['label_ordering'].astype(int)

# get attributes

In [7]:
df_attributes = pd.read_csv('../attribute_extraction_metadata_template/attribute_definition_top25L2_filtered_augmented_redacted_addedsinglemulti_01182023.csv')

df_attributes_group = df_attributes.groupby('attribute_field').agg({
    'category_attributevalue': lambda x: reduce(lambda x, y: x+y, [eval(i) for i in x])
}).reset_index()

In [8]:
len(df_attributes_group), df_attributes_group.category_attributevalue.apply(len).sum()

(230, 516378)

In [9]:
attributes = set(df_attributes_group['attribute_field'])

In [10]:
category_paths = set(df_attributes['category'])

In [11]:
more_paths = []
for i in category_paths:
    tmp = []
    for j in i.split(' > '):
        tmp.append(j)
        more_paths.append(' > '.join(tmp))
category_paths.update(more_paths)

# existing appen data

In [13]:
# df1 = pd.read_csv('appen/output_batch_correct_v3/product_attribution_till_030923_valid_units.csv')
# df2 = pd.read_csv('appen/output_batch_correct_v4/f2072154_f2075355_post_03.13.23_cleaned_invalid.csv')
# df3 = pd.read_csv('appen/output_batch_correct_v4/f2072154_f2075355_post_03.13.23_cleaned_valid-empty units.csv')
# df4 = pd.read_csv('appen/output_batch_correct_v4/Product Attribution_03.10.23_Batch2_highimpression.csv')
# df5 = pd.read_csv('appen/output_batch_correct_v4/product_attribution_03.03.23_invalid_units_03.02.23_Rework.csv')

In [14]:
# len(df1), len(df2), len(df3), len(df4), len(df5)

(38076, 866, 19678, 10808, 419)

In [15]:
# df = pd.concat([df1, df2, df3, df4, df5])

In [12]:
df = pd.read_csv('appen/output_batch_correct_v6/Product Att_03.17.23_Batch2.csv')

In [13]:
df_uniq = df.drop_duplicates('label_ordering', keep='first')

In [14]:
len(df), len(df_uniq)

(51258, 51258)

In [20]:
df_uniq_merge = df_uniq.merge(df_original[['label_ordering', 'title', 'product_description']].rename(
    columns={'title': 'title_original', 'product_description': 'product_description_original'}), on='label_ordering', how='left')

In [21]:
len(df_uniq_merge)

51258

In [22]:
df_uniq_merge[df_uniq_merge.title_original.isna()]

,label_ordering,sample_method,product_id,title,product_description,main_image_url,product_category,l2_category,rater1_no_attributes,rater1_unlisted_value,rater1_output,rater2_no_attributes,rater2_unlisted_value,rater2_output,rater3_no_attributes,rater3_unlisted_value,rater3_output,title_original,product_description_original


In [23]:
df_uniq_merge[df_uniq_merge.product_description_original.isna()]

,label_ordering,sample_method,product_id,title,product_description,main_image_url,product_category,l2_category,rater1_no_attributes,rater1_unlisted_value,rater1_output,rater2_no_attributes,rater2_unlisted_value,rater2_output,rater3_no_attributes,rater3_unlisted_value,rater3_output,title_original,product_description_original


In [27]:
df_uniq_merge['title'] = df_uniq_merge['title_original']
df_uniq_merge['product_description'] = df_uniq_merge['product_description_original']

In [31]:
df_uniq_merge['final_output'] = df_uniq_merge['rater1_output'].fillna('') + '\n' + df_uniq_merge['rater2_output'].fillna('') + '\n' + \
    df_uniq_merge['rater3_output'].fillna('')

In [32]:
correct = []
errors = []
errors_fixed = []
nonempty = []
empty = []
for i in df_uniq_merge.to_dict('records'):
    res = [] if ((not isinstance(i['final_output'], str)) and (np.isnan(i['final_output']))) else i['final_output'].split('\n')
    res = sorted(set(res))
    res = [i for i in res if len(i) > 0]
    if len(res) == 0:
        empty.append(i)
    is_correct = True
    for j in res:
        if len(j.split(' > ')) < 2:
            i['explanation'] = f"{j} has no attribute name"
            errors.append(i)
            is_correct = False
            break
        if j.split(' > ')[-2] not in attributes:
            i['explanation'] = f"{j} has invalid attribute name, because {j.split(' > ')[-2]} is not a valid attribute name"
            errors.append(i)
            is_correct = False
            break
    if is_correct:
        correct.append(i)
        if len(res) > 0:
            nonempty.append(i)
for i in errors:
    res = [] if ((not isinstance(i['final_output'], str)) and (np.isnan(i['final_output']))) else i['final_output'].split('\n')
    res = sorted(set(res))
    res = [i for i in res if len(i) > 0]
    corrected_res = []
    for j in res:
        if j.split(' > ')[-2] in attributes:
            corrected_res.append(j)
    i['final_output_corrected'] = '\n'.join(corrected_res)
    errors_fixed.append(i)

len(df), len(correct), len(errors), len(errors_fixed), len(empty), len(nonempty), len(correct) + len(errors)

(51258, 50346, 912, 912, 6821, 43525, 51258)

In [33]:
len(empty) / len(correct), len(nonempty) / len(correct), len(errors) / len(df)

(0.13548246136733802, 0.864517538632662, 0.017792344609621914)

In [34]:
df_correct = pd.DataFrame(correct)
df_error = pd.DataFrame(errors)
df_error_fixed = pd.DataFrame(errors_fixed)

In [35]:
df_nonempty = pd.DataFrame(nonempty)
df_empty = pd.DataFrame(empty)

In [36]:
assert len(df_correct) == len(df_empty) + len(df_nonempty)

In [37]:
df_empty.rater3_no_attributes.all()

True

In [38]:
len(df_empty), len(df_nonempty), len(df_error)

(6821, 43525, 912)

In [39]:
df_nonempty.to_csv('appen/output_batch_correct_v6/product_attribution_just_032723_valid_units.csv', index=False)
df_empty.to_csv('appen/output_batch_correct_v6/product_attribution_just_032723_empty_units.csv', index=False)
df_error.to_csv('appen/output_batch_correct_v6/product_attribution_just_032723_invalid_units.csv', index=False)

In [40]:
df_error_fixed.to_csv('appen/output_batch_correct_v6/product_attribution_just_032723_invalid_units_fixed.csv', index=False)

In [41]:
df_empty.sample(1).to_dict()

{'label_ordering': {1575: 65506},
 'sample_method': {1575: 'only_text'},
 'product_id': {1575: '6178cd87cd516376b43d159f'},
 'title': {1575: "10 PCS/Set Professional Japanese Red Handle Chef's Knife Meat Cutter Laser Damascus Pattern Chef's Knife Boning Bread Knife Set kitchen knives kitchen tools"},
 'product_description': {1575: "10 PCS/Set Professional Japanese Red Handle Chef's Knife Meat Cutter Laser Damascus Pattern Chef's Knife Boning Bread Knife Set kitchen knives kitchen tools\n\nFeature：\n\n[Name]: Kitchen Knife\n[Blade]: 7Cr17\n[Features]: sharp and durable; environmental health; lightweight effort\nFeatures and Scope:\nThe hardened state, hard, but higher than 8Cr17,11Cr17 toughness. Used as cutting tools, measuring tools, bearings.\n\nWith it, your kitchen is more complete!\n\n8 inch Chef knife\n8 inch Bread knife\n8 inch Utility knife\n7.5 inch Kiritsuke knife\n7 inch Cleaver knife\n7 inch Santoku knife\n6 inch Boning knife\n5 inch Santoku knife\n5 inch Utility knife\n3.5

In [42]:
df_nonempty.sample(1).to_dict()

{'label_ordering': {9975: 62940},
 'sample_method': {9975: 'text_and_img'},
 'product_id': {9975: '60d3e75fcd06b916da2666fa'},
 'title': {9975: '2021Upgraded 1/2/3PCS Non-Woven Folding Storage Box Quilt Storage Bag Wardrobe Clothing Storage Box Dustproof Three Grid Visible Organizer Bag Home Foldable Storage Organization'},
 'product_description': {9975: '【Features】:\n\n- Convenient storage bags for storing and protecting clothing and household items.\n\n- For seasonal clothing, blankets, linen, festive decorations, toys, photos, baby clothing, saris and more.\n\n- Ideal for families, college dorms, closets, bedrooms, basement, loft, under-bed storage and more.\n\n- Carry lifting lift bags for stacking or transporting from one place to another - Easily move to shelves, closets, etc. - Quickly pack/unpack in seasons, travel, move or as needed\n\n- Fold or fold under the bed or drawer when not in use.\n\n- Zip closure protects against dust, insects, water spots and odors.\n\n- Made of po

In [43]:
df_error.sample(1).to_dict()

{'label_ordering': {785: 92320},
 'sample_method': {785: 'only_text'},
 'product_id': {785: '61755ffd643c78b313c045aa'},
 'title': {785: '  Heating Fan Practical Anti-scald ABS  Lightweight    '},
 'product_description': {785: 'Specifications: \nSince it has a small size and light weight, this product is easy to carry and change place, which can use at night during sleep with its design of silent operation.\nIt is easy to carry and change place with its small size and light weight, which can use at night during sleep since it is silent operated.\nAs it has a design of silent operation, this product can use at night during sleep, which is easy to carry and change place with its small size and light weight.\nDesigned with its silent operation, this product can use at night during sleep, which is easy to carry and change place with its small size and light weight.\nThis product is can be used at night during sleep with its design of silent operation, featuring its light weight and small s

In [44]:
df_error_fixed.sample(1).to_dict()

{'label_ordering': {581: 70080},
 'sample_method': {581: 'only_text'},
 'product_id': {581: '605a0eff1f19f92a2690240b'},
 'title': {581: "Fashion Light-weight Men Leisure Shoes Suede Loafers Men's Driving Style Loafers Size 39-46"},
 'product_description': {581: "Welcome to our store, wish you have a nice shopping!\n100% brand new high quality\n\nBecause the size of the product is different, so the cost is also different,\nsmall size is cheaper than big size, please understand! thank you!We mark the\nprice of the smallest size in the picture!\nTip: If you like to wear loose shoes, please choose one size larger than usual.\n\nBecause US shoe size and China shoe size is not completely equal, so the following shoe size is only for reference:(Best selection method: check foot length)\n\nUS 6 = EU 38 =foot length 23.5 - 24 cm\nUS 6.5 = EU 39 =foot length 24 - 24.5 cm\nUS 7 = EU 40 =foot length 24.5 - 25 cm\nUS 8 = EU 41 =foot length 25 - 25.5 cm\nUS 8.5 = EU 42 =foot length 25.5 - 26 cm\nUS

In [45]:
leftover = set(df_original.label_ordering) - set(df_uniq_merge.label_ordering)
df_leftover = df_original[df_original.label_ordering.apply(lambda x: x in leftover)]

In [46]:
len(df_leftover)

38

In [47]:
df_leftover.to_csv('appen/output_batch_correct_v6/product_attribution_missed_just_032723.csv', index=False)